# Project 4 - Background Work

## Setting Up

In [11]:
import os
import pandas as pd
import petl as etl

In [12]:
raw_data_path = "data/raw"
raw_data_directory = []

for filename in os.listdir(raw_data_path):
    file = os.path.join(raw_data_path, filename)

    if os.path.isfile(file):
        #print(file)
        raw_data_directory.append(file)

In [13]:
raw_data_directory

['data/raw\\2010-2020_Bassetlaw_outlet.csv',
 'data/raw\\2010-2020_Castle_Point_outlet.csv',
 'data/raw\\2010-2020_Ceredigion_store.json',
 'data/raw\\2010-2020_Cheshire_East_outlet.csv',
 'data/raw\\2010-2020_Chiltern_store.json',
 'data/raw\\2010-2020_Dorset_outlet.json',
 'data/raw\\2010-2020_East_Cambridgeshire_outlet.csv',
 'data/raw\\2010-2020_East_Devon_branch.csv',
 'data/raw\\2010-2020_East_Sussex_outlet.csv',
 'data/raw\\2010-2020_Epsom_and_Ewell_store.json',
 'data/raw\\2010-2020_Flintshire_branch.csv',
 'data/raw\\2010-2020_Forest_of_Dean_store.csv',
 'data/raw\\2010-2020_Fylde_outlet.csv',
 'data/raw\\2010-2020_Greater_London_branch.csv',
 'data/raw\\2010-2020_Greater_London_branch.json',
 'data/raw\\2010-2020_Haringey_store.json',
 'data/raw\\2010-2020_Kent_branch.json',
 'data/raw\\2010-2020_Lambeth_store.json',
 'data/raw\\2010-2020_Larne_store.json',
 'data/raw\\2010-2020_Lichfield_outlet.csv',
 'data/raw\\2010-2020_Lincoln_branch.csv',
 'data/raw\\2010-2020_Mole_Valle

In [14]:
len(raw_data_directory)

94

In [15]:
for file in raw_data_directory:
    if str(file).endswith('json'):
        if str(file).replace('json', 'csv') in raw_data_directory:
            raw_data_directory.remove(str(file).replace('json', 'csv'))

len(raw_data_directory)

86

In [16]:
raw_data_directory

['data/raw\\2010-2020_Bassetlaw_outlet.csv',
 'data/raw\\2010-2020_Castle_Point_outlet.csv',
 'data/raw\\2010-2020_Ceredigion_store.json',
 'data/raw\\2010-2020_Cheshire_East_outlet.csv',
 'data/raw\\2010-2020_Chiltern_store.json',
 'data/raw\\2010-2020_Dorset_outlet.json',
 'data/raw\\2010-2020_East_Cambridgeshire_outlet.csv',
 'data/raw\\2010-2020_East_Devon_branch.csv',
 'data/raw\\2010-2020_East_Sussex_outlet.csv',
 'data/raw\\2010-2020_Epsom_and_Ewell_store.json',
 'data/raw\\2010-2020_Flintshire_branch.csv',
 'data/raw\\2010-2020_Forest_of_Dean_store.csv',
 'data/raw\\2010-2020_Fylde_outlet.csv',
 'data/raw\\2010-2020_Greater_London_branch.json',
 'data/raw\\2010-2020_Haringey_store.json',
 'data/raw\\2010-2020_Kent_branch.json',
 'data/raw\\2010-2020_Lambeth_store.json',
 'data/raw\\2010-2020_Larne_store.json',
 'data/raw\\2010-2020_Lichfield_outlet.csv',
 'data/raw\\2010-2020_Lincoln_branch.csv',
 'data/raw\\2010-2020_Mole_Valley_store.csv',
 'data/raw\\2010-2020_Newham_branch.

In [17]:
# acquire product list from directory
products_table = etl.fromcsv('data/other/products_list.csv')
# cut wanted fields
products_table = etl.cut(products_table, 'product', 'category')

products_table = etl.values(products_table, 'product', 'category')

product_list = list(products_table)

# print for checking
product_list


[('Fairtrade Bananas Loose', 'fruits & vegetables'),
 ('British carrots loose', 'fruits & vegetables'),
 ('Onions Loose', 'fruits & vegetables'),
 ('British baking potatoes loose', 'fruits & vegetables'),
 ('Red pepper', 'fruits & vegetables'),
 ('Mixed pepper', 'fruits & vegetables'),
 ('Brocolli loose', 'fruits & vegetables'),
 ('Lemon', 'fruits & vegetables'),
 ('Spring onions bunch', 'fruits & vegetables'),
 ('Sweet potatoes loose', 'fruits & vegetables'),
 ('courgette loose', 'fruits & vegetables'),
 ('baby potatoes', 'fruits & vegetables'),
 ('british parsnips loose', 'fruits & vegetables'),
 ('fine beans', 'fruits & vegetables'),
 ('garlic', 'fruits & vegetables'),
 ('celery', 'fruits & vegetables'),
 ('aubergine', 'fruits & vegetables'),
 ('raspberries', 'fruits & vegetables'),
 ('british bramley cooking apples', 'fruits & vegetables'),
 ('easy peeler loose', 'fruits & vegetables'),
 ('scottish salmon', 'meat & fish'),
 ('beef mince', 'meat & fish'),
 ('british fresh chicken br

## Cleanup & Transform Functions

In [18]:
# cleanup function
def table_cleanup(table):
    return print(table)

# add product category function
def add_prod_cat(prv, cur, nxt):
    for row in product_list:
        if cur.product == row[0]:
            return row[1]

## Iterating Through Files

In [21]:
## for each file in the directory
#for file in raw_data_directory:
## check file type
    #if str(file).endswith('csv'):
## extract from said type
        #current_table = etl.fromcsv(file)
## initiate cleanup functions
        #table_cleanup(current_table)
    #else:
        #current_table = etl.fromjson(file)
## initiate cleanup functions
        #table_cleanup(current_table)


current_table = etl.fromcsv(raw_data_directory[0])

print(type(current_table))


# remove unwanted fields
current_table = etl.cutout(
                    current_table, 
                    'month', 'day', 'hour')

# convert fields to int & float
current_table = etl.convert(
                    current_table, 
                    'quantity_purchased',
                    int)
# round up field to 2 decimal points
current_table = etl.convert(
                    current_table, 
                    'amount_in_gbp',
                    lambda cell: round(float(cell), 2))

# if table has the header sku or item.. change to product
if 'sku' in etl.header(current_table):
    current_table = etl.rename(
                        current_table, 
                        'sku', 'product')
elif 'item' in etl.header(current_table):
    current_table = etl.rename(
                        current_table, 
                        'item', 'product')

# rename header to only have the one name (total_quantity_purchase)
if 'quantity' in etl.header(current_table):
    current_table = etl.rename(
                        current_table,
                        'quantity', 'total_quantity_purchased'
    )
elif 'total_quantity' in etl.header(current_table):
    current_table = etl.rename(
                        current_table,
                        'total_quantity', 'total_quantity_purchased'
    )
elif 'quantity_purchased' in etl.header(current_table):
    current_table = etl.rename(
                        current_table,
                        'quantity_purchased', 'total_quantity_purchased'
    )

print(type(current_table))
# adding product categories to table
current_table = etl.addfieldusingcontext(current_table, 'product_category', add_prod_cat)

print(type(current_table))


# obtain city name
city_name = str(raw_data_directory[0]).split('_', 1).pop(1).rsplit('.').pop(0)
# adding city as a field
current_table = etl.addfield(current_table, 'city', city_name)

print(type(current_table))


current_table



<class 'petl.io.csv_py3.CSVView'>
<class 'petl.transform.headers.RenameView'>
<class 'petl.transform.basics.AddFieldUsingContextView'>
<class 'petl.transform.basics.AddFieldView'>


year,product,total_quantity_purchased,amount_in_gbp,product_category,city
2010,chocolate twist,182,145.6,bakery,Bassetlaw_outlet
2010,plastic sunshades,98,313.6,clothing,Bassetlaw_outlet
2010,lg washing machine,151,12080.0,appliances,Bassetlaw_outlet
2010,fresh whole chicken,230,257.6,meat & fish,Bassetlaw_outlet
2010,free range large eggs x12,150,180.0,dairy,Bassetlaw_outlet


In [20]:
# change to DataFrame
type(current_table)
test = etl.fromcsv(raw_data_directory[0])
type(test)

petl.io.csv_py3.CSVView

In [27]:
file_name = str(raw_data_directory[0]).split('\\').pop(1)
etl.tocsv(current_table, f'data/refined/{file_name}')



,year,product,total_quantity_purchased,amount_in_gbp,product_category,city
0,2010,chocolate twist,182.0,145.6,bakery,Bassetlaw_outlet
1,2010,plastic sunshades,98.0,313.6,clothing,Bassetlaw_outlet
2,2010,lg washing machine,151.0,12080.0,appliances,Bassetlaw_outlet
3,2010,fresh whole chicken,230.0,257.6,meat & fish,Bassetlaw_outlet
4,2010,free range large eggs x12,150.0,180.0,dairy,Bassetlaw_outlet
...,...,...,...,...,...,...
1119657,2020,chicken sausages,225.0,450.0,meat & fish,Bassetlaw_outlet
1119658,2020,silk scarf,8.0,1072.0,clothing,Bassetlaw_outlet
1119659,2020,denim jeams,185.0,5550.0,clothing,Bassetlaw_outlet
1119660,2020,cod fillets,48.0,96.0,meat & fish,Bassetlaw_outlet


In [28]:
example_df = pd.read_csv(f'data/refined/{file_name}')

example_df

,year,product,total_quantity_purchased,amount_in_gbp,product_category,city
0,2010,chocolate twist,182.0,145.6,bakery,Bassetlaw_outlet
1,2010,plastic sunshades,98.0,313.6,clothing,Bassetlaw_outlet
2,2010,lg washing machine,151.0,12080.0,appliances,Bassetlaw_outlet
3,2010,fresh whole chicken,230.0,257.6,meat & fish,Bassetlaw_outlet
4,2010,free range large eggs x12,150.0,180.0,dairy,Bassetlaw_outlet
...,...,...,...,...,...,...
1119657,2020,chicken sausages,225.0,450.0,meat & fish,Bassetlaw_outlet
1119658,2020,silk scarf,8.0,1072.0,clothing,Bassetlaw_outlet
1119659,2020,denim jeams,185.0,5550.0,clothing,Bassetlaw_outlet
1119660,2020,cod fillets,48.0,96.0,meat & fish,Bassetlaw_outlet


In [29]:
test_df = etl.todataframe(current_table)

In [30]:
test_df

,year,product,total_quantity_purchased,amount_in_gbp,product_category,city
0,2010,chocolate twist,182.0,145.6,bakery,Bassetlaw_outlet
1,2010,plastic sunshades,98.0,313.6,clothing,Bassetlaw_outlet
2,2010,lg washing machine,151.0,12080.0,appliances,Bassetlaw_outlet
3,2010,fresh whole chicken,230.0,257.6,meat & fish,Bassetlaw_outlet
4,2010,free range large eggs x12,150.0,180.0,dairy,Bassetlaw_outlet
...,...,...,...,...,...,...
1119657,2020,chicken sausages,225.0,450.0,meat & fish,Bassetlaw_outlet
1119658,2020,silk scarf,8.0,1072.0,clothing,Bassetlaw_outlet
1119659,2020,denim jeams,185.0,5550.0,clothing,Bassetlaw_outlet
1119660,2020,cod fillets,48.0,96.0,meat & fish,Bassetlaw_outlet


In [33]:
test_df.dtypes

year                         object
product                      object
total_quantity_purchased    float64
amount_in_gbp               float64
product_category             object
city                         object
dtype: object